In [11]:
class Epidemy(Graph):
    '''Epidemy extends the igraph's class Graph. Ciao
    The additional functionalities are:
    
    1 Built-in getters for graph metrics
        1.1 Plotting
            a)
            b)
        1.2 Metrics
            a) distribution model
            b) eggr
    
    '''
    graph = None
    patient_zero = None
    sentinels = None
    global I
    
    def __init__(self, graph_edge_list, patient_zero = None, sentinels = None):
        '''The compartment label is a byte that can take values 0, 1 and 2, indicating respectivly
        0 - Suscebtible node
        1 - Infected node'''
        
        
        self.graph = Graph.Read_Ncol(graph_edge_list, directed=False)
        self.patient_zero = patient_zero
        self.sentinels = sentinels
             
    
    #setters  
    def setPatientZero(self, patient_zero):
        if patient_zero == None:
            self.patient_zero = None
        else:
            self.patient_zero = set(patient_zero)
        
    def setSentinels(self, sentinels):
        if sentinels == None:
            self.sentinels = None
        else:
            self.sentinels = set(sentinels)
        
    def resetSentinels(self):
        self.graph.vs["iteration"] = np.nan
        
    def resetNodes(self):
        self.graph.vs["compartment"] = np.zeros(len(self.graph.vs), dtype = np.uint8)
    
     
    #getters
    def getDegreeDistribution(self):
        x,y = np.unique(self.graph.degree(), return_counts=True)
        print(len(self.graph.degree()))
        return (x,y)
    
    def getMaxDegreeVertex(self):
        return self.graph.vs[self.graph.degree().index(self.graph.maxdegree())]
    
    def getSentinelsIteration(self):
        return [i["iteration"] for i in self.graph.vs[self.sentinels]]

                
    def getRankingCombinations(self, number):
        aux = np.transpose(sorted(np.column_stack((a.graph.vs.indices, self.graph.degree())), key=lambda x: x[1]))[0]
        rankings = [(aux[:number], "worst_degree"), (aux[-number:], "best_centrality")]
        aux = np.transpose(sorted(np.column_stack((a.graph.vs.indices, self.graph.closeness())), key=lambda x: x[1]))[0].astype(int)
        rankings += [(aux[:number], "worst_closeness"), (aux[-number:], "best_closeness")]
        aux = np.transpose(sorted(np.column_stack((a.graph.vs.indices, self.graph.betweenness(directed=False))), key=lambda x: x[1]))[0].astype(int)        
        rankings += [(aux[:number], "worst_betweenness"), (aux[-number:], "best_betweenness")] 
        aux = np.transpose(sorted(np.column_stack((a.graph.vs.indices, self.graph.pagerank(directed=False))), key=lambda x: x[1]))[0].astype(int)
        rankings += [(aux[:number], "worst_pagerank"), (aux[-number:], "best_pagerank")]  
        return itertools.permutations(rankings, 2)
        
    
    #Epidemy Utilities
    def printDegreeDistribution(self, loglog=False):
        '''Prints the degree distirbution of the underlying network using a logarithmic scale'''
        x,y = self.getDegreeDistribution()
        if loglog:
            plt.scatter(np.log(x), np.log(y))
        else:
            plt.scatter(x, y)
        plt.show()
    
    def getRankings(self):
        try:
            rankings = hkl.load(dataset[:-4]+"_rankings.hkl")
            print 1111111111111
        except:
            rankings = list(self.getRankingCombinations(10))
            hkl.dump(rankings, dataset[:-4]+"_rankings.hkl")
        return rankings

    def SIRRankedSimulation(self):
        df = pd.DataFrame([])
        for ranking in self.getRankings():
            seq = []
            print("Patient_zero:", ranking[0][1], "Sentinels: ", ranking[1][1])
            df[str(ranking[0][1])+"_"+str(ranking[1][1])] = self.N_SIR(patient_zero = ranking[0][0], sentinels = ranking[1][0])
        return df
    
    def N_SIR(self, num_simulation = 10, **kwargs):
        seq = []
        for i in range(num_simulation):
            print("Simulation number", i+1)
            self.SIR(**kwargs)
            seq = seq + [np.nan if pd.isnull(v["iteration"]) else int(v["iteration"]) for v in self.graph.vs[a.sentinels]]
        return seq
            
        
    #Epidemic Models
    def SIR(self, beta = 0.5, mu = 0.1,
            patient_zero = None, num_pat = 5,
            sentinels = None, num_sen = 5,
            friend_paradox = False, verbose= False):
        """Simulate an epidemy outbreaks using a sir model"""
        
       
    
        if (self.sentinels is None) & (sentinels is None) :
            self.setSentinels(np.random.choice(self.graph.vs.indices, size=num_sen, replace=False))
            print("No sentinels detected. Random selection: ", self.sentinels)
        elif sentinels is not None:
            self.setSentinels(sentinels)
            
        #print("sentinels", self.sentinels)
        
        if friend_paradox:
            I = list(chain.from_iterable([self.graph.neighbors(v) for v in self.graph.vs[self.sentinels]]))
            I = set(np.random.choice(I, size =num_pat, replace=False))
        elif patient_zero is not None:
            I = set(patient_zero)
        elif self.patient_zero is not None:
            I = set(self.patient_zero)
        else:
            I = set(np.random.choice(self.graph.vs.indices, size= 5,replace =False))
            print("No patatient zero detected. Random selection: ", I)
        
        self.resetNodes()
        self.resetSentinels();
        
        development = []
        if verbose:
            print("Starting SIR simulation", "beta:", beta, "mu", mu,
              "sentinels:", self.sentinels, "patient_zero: ", self.patient_zero, "friend_paradox", friend_paradox)
        
        for iteration in itertools.count():
            if(len(I)==0):
                break
                
            if verbose:   
                print("Nodes in the Infected Compartment: ",len(I))
            
            dI = set([j for j in list(itertools.chain.from_iterable(self.graph.neighborhood(I)))
                      if self.graph.vs[j]["compartment"] == 0 and np.random.random() < beta])
            self.graph.vs[dI]["compartment"] = 1 


            # Finds new removed nodes and update the status
            dR = set([k for k in I if np.random.random() < mu])
            self.graph.vs[dR]["compartment"] = 2
            
            self.graph.vs[dI & set(self.sentinels)]["iteration"] = str(iteration)
            

            I = (I | dI) - (dR)
            
            development +=  [len(I)]
        return development

dataset = "facebook.txt"
a = Epidemy(dataset)
#a.printDegreeDistribution(loglog = True) 
#q = a.SIRRankedSimulation()

Orkut is a free on-line social network where users form friendship each other. Orkut also allows users form a group which other members can then join. We consider such user-defined groups as ground-truth communities. We provide the Orkut friendship social network and ground-truth communities. This data is provided by Alan Mislove et al.

We regard each connected component in a group as a separate ground-truth community. We remove the ground-truth communities which have less than 3 nodes. We also provide the top 5,000 communities with highest quality which are described in our paper. As for the network, we provide the largest connected component.



The graph is undirected.

In [4]:
from igraph import *
import numpy as np
import matplotlib.pyplot as plt
import itertools
import seaborn as sb
import pandas as pd
from itertools import chain
import itertools
import pandas as pd
import hickle as hkl 


# Digital Epidemiology Assignment 1

In this homework we are going to use a real social netwrok in order to simulate and analyze a SIR epidemy. 

The network that we are going to use is Orkut's social graph. I decided to use this dataset because Orkut was a community base social network. Since its connections are intrests-based, this would allow us to simulate our empidemy on a network based on a real-social interactions.

Orkut's social graph is undirected.

## Part 1 
 
I decided to implement an extende class of igraph librery called Epidemy. This library is freerly aviable for download from the following link : fefefaeffez

However to make things easier for the correction I included my package in this folder under the name Epidemy.py.

Essentially the epidemy class implements the following functionalities:

* Create Epidemy Object given a Network and an Epidemic Model
* Simulate on the given network the given Epidemic Model
* Get Network metrics
    * Degree Score
    * Closeness Score
    * Betwenness Score
    * Pagerank Score


In this way we can logically separate our code needed to perform some analysis from the analys itself. If the revisor is intrested in the actual implementation of the algorithm she can check the bottom of the ipython notebook to see the implementation.



### 1.1 Compute and Plot a Social Network Degree [X]


In this section we are going to load in memory Orkut and plot its degree distribution.

In [ ]:
dataset = "facebook.txt"
a = Epidemy(dataset)
#a.printDegreeDistribution(loglog = True)

### 1.2 SIR Epidemic Model Simulation [X]

Now we are going to perform a single simulatation of a SIR epidemy on the graph:

In [ ]:
epidemic_curve = a.SIR(beta = 0.9, mu = 0.1, patient_zero=a.graph.vs.indices[-10:])
plt.plot(epidemic_curve)
plt.show()

## Part 2

In this section we are going to have a better understaing of the epidemic cycle. More specifically we are going to use epdiemy sentinels, which are a random set o N nodes of the graph which will record the arrival time to them.
Epidemic sentinels are crucial in order to understand the empidemic outbreaks, because are the one that 

### 2.1 SIR Simulation with Static Sentinels [ ] 

In this subsection we are going to exploit the sentinel functionality provided by the class Epidemy. In the previous section we already run a SIR simulation using sentinels, we can retrive the information stored simply accessing the sentinel field of 

In [ ]:
df1 = pd.DataFrame([])
a.N_SIR(beta = 0.9, mu = 0.1)             
df1 = pd.DataFrame(a.N_SIR(num_simulation=100, beta = 0.9, mu = 0.1), columns = ["beta_0.9_mu_0.1"])  
sb.boxplot(df1)
plt.show()

### 2.2 SIR Simulation with Dynamic Random Sentinels, Seeds and Parameters [ ]

Now we want to extend the previous analysis testing different combination of 

* SIR parameters ($\mu$, $\beta$)
* Different sentinels at each loop
* Different number of sentinels
* No overlap between seed and sentinels
* Different seed at each loop
* Multiple Seeds

Most of those features are easily simulable thanks via the built-in functionalities of the epidemy class.

In [9]:

df1 = pd.DataFrame([])
for i in range(20):
    rnd_beta, rnd_mu = np.random.random(),np.random.random()
    rnd_pat, rnd_sen = np.random.randint(1,a.graph.vcount()),np.random.randint(1,a.graph.vcount())
    simulation_name = "_".join(map(lambda x: str(x),[rnd_beta,rnd_mu,rnd_pat,rnd_sen])[:-1])                                                                       
    df1[simulation_name] = a.N_SIR(num_simulation=10, beta = rnd_beta, mu = rnd_mu,
                  patient_zero= None, num_pat = rnd_pat,
                  sentinels = None, num_sen = rnd_sen)  
sb.boxplot(df1)
plt.show()


('Simulation number', 1)
('No sentinels detected. Random selection: ', set([3232, 1828, 3560, 553, 1706, 1303, 3101, 1240, 527, 4029, 1161, 760, 1242, 3087, 1756, 2205]))
('No patatient zero detected. Random selection: ', set([2840, 761, 228, 1256, 3429]))


/home/aalto/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:38: FutureWarning: comparison to `None` will result in an elementwise object comparison in the future.


('Simulation number', 2)
('No patatient zero detected. Random selection: ', set([2008, 283, 354, 3483, 445]))
('Simulation number', 3)
('No patatient zero detected. Random selection: ', set([272, 1898, 2317, 2698, 2471]))
('Simulation number', 4)
('No patatient zero detected. Random selection: ', set([1468, 2546, 1548, 1077, 3894]))
('Simulation number', 5)
('No patatient zero detected. Random selection: ', set([2537, 3627, 1669, 222, 2133]))
('Simulation number', 6)
('No patatient zero detected. Random selection: ', set([2147, 819, 1293, 3189, 149]))
('Simulation number', 7)
('No patatient zero detected. Random selection: ', set([1680, 3480, 1522, 1044, 990]))
('Simulation number', 8)
('No patatient zero detected. Random selection: ', set([1656, 1577, 1162, 3291, 68]))
('Simulation number', 9)
('No patatient zero detected. Random selection: ', set([64, 514, 629, 566, 2535]))
('Simulation number', 10)
('No patatient zero detected. Random selection: ', set([8, 3760, 971, 181, 197]))
('S

/home/aalto/anaconda2/lib/python2.7/site-packages/seaborn/categorical.py:2171: UserWarning: The boxplot API has been changed. Attempting to adjust your arguments for the new API (which might not work). Please update your code. See the version 0.6 release notes for more info.
  warnings.warn(msg, UserWarning)


In [ ]:
seq = [-1 if pd.isnull(elem) else int(elem) for elem in seq]
seq

### 2.3 SIR Simulation with Static Top-Ranked Sentinels [ ]

Now we want to investigate the impact of specific centrality metrics in the choice of 

In [ ]:
q = a.SIRRankedSimulation()

sb.boxplot(df)
plt.show()

1111111111111
('Patient_zero:', 'worst_degree', 'Sentinels: ', 'best_centrality')
('Simulation number', 1)


/home/aalto/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:38: FutureWarning: comparison to `None` will result in an elementwise object comparison in the future.


('Simulation number', 2)
('Simulation number', 3)
('Simulation number', 4)
('Simulation number', 5)
('Simulation number', 6)
('Simulation number', 7)
('Simulation number', 8)
('Simulation number', 9)
('Simulation number', 10)
('Patient_zero:', 'worst_degree', 'Sentinels: ', 'worst_closeness')
('Simulation number', 1)
('Simulation number', 2)
('Simulation number', 3)
('Simulation number', 4)
('Simulation number', 5)
('Simulation number', 6)
('Simulation number', 7)
('Simulation number', 8)
('Simulation number', 9)
('Simulation number', 10)
('Patient_zero:', 'worst_degree', 'Sentinels: ', 'best_closeness')
('Simulation number', 1)
('Simulation number', 2)
('Simulation number', 3)
('Simulation number', 4)
('Simulation number', 5)
('Simulation number', 6)
('Simulation number', 7)
('Simulation number', 8)
('Simulation number', 9)
('Simulation number', 10)
('Patient_zero:', 'worst_degree', 'Sentinels: ', 'worst_betweenness')
('Simulation number', 1)
('Simulation number', 2)
('Simulation numb

In [ ]:
print a.getBestVertexByCentrality(10)

## Part 3 [ ]

In this section we are intrested in epidemy detection with no global topology information.

### 3.1 Friend Paradox and Local Centrality Measures [ ]

In this subsection we are intrested in minimizing the **detection time** of our epidemy by the sentinels. We suppose that we do not have any information about the global topology of the graph, but only the informations about the neighbours of our seed nodes.

In order to achive this result we are going to exploit the so called **friend paradox**, which we are going to discuss in section 3.2

$\mu=\frac{\sum_{v\in V} d(v)}{|V|}=\frac{2|E|}{|V|}.$

$\frac{\sum_{v\in V} d(v)^2}{2|E|}=\mu + \frac{\sigma^2}{\mu},$

where $ {\sigma}^{2} $ is the variance of the degrees in the graph

In [ ]:
seq = []
for i in range(2):
    a.setSentinels(None)
    a.setSentinels(None)
    a.SIR(beta = 0.9, mu = 0.1, friend_paradox = True , verbose = True)
    seq = seq + [np.nan if pd.isnull(v["iteration"]) else int(v["iteration"]) for v in a.graph.vs[a.sentinels]]
sb.boxplot(np.array(seq))
plt.show()

### 3.2 Why the Friendship Paradox Works*? (and also the local centrality measures) [ ]

The Friendship Paradox states that on average given a node in a graph (that we can imagine as a real person), on average its negihbours (that we can think as its friend) will have an higher degree than the node itself. Exploiting this characteristic is possible to choose from our neighbours the ones that have an higher degree randomly being sure that on average those nodes will have a higher degree, hence an higher porbability to detect the epidemy out break.

We can mix this approach using a also some metrics nalysis an 

## Part 4 (Epidemy Class) [ ]

The epidemy class is the core of this notebook. It extends the igraph graph class adding the methods required to premor an epidemy analysis smoothly

In [ ]:
planets = sns.load_dataset("planets")
planets

In [ ]:
import numpy as np
import seaborn as sns
sns.set(style="ticks", palette="muted", color_codes=True)

# Load the example planets dataset
planets = sns.load_dataset("planets")

# Plot the orbital period with horizontal boxes
ax = sns.boxplot(x="distance", y="method", data=planets,
                 whis=np.inf, color="c")

# Add in points to show each observation
sns.stripplot(x="distance", y="method", data=planets,
              jitter=True, size=3, color=".3", linewidth=0)


# Make the quantitative axis logarithmic
ax.set_xscale("log")
sns.despine(trim=True)
plt.show()

In [ ]:
 def getBestVertexByCentrality(self, number):
        aux = sorted(self.graph.degree())
        return (aux[:number], "worst_degree", aux[-number:], "best_centrality")
        
    def getBestVertexByCloseness(self, number):
        aux = sorted(self.graph.closeness())
        return (aux[:number], "worst_closeness", aux[-number:], "best_closeness")
        
    def getBestVertexByBetweennes(self, number):
        aux = sorted(self.graph.pagerank())
        return (aux[:number], "worst_betweenness", aux[-number:], "best_betweenness"]
    
    def getBestVertexByPagerank(self, number):
        aux = sorted(self.graph.pagerank(directed=False))
        return ((aux[:number], "worst_pagerank"), (aux[-number:], "best_pagerank")]

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# fake up some data
spread = np.random.rand(50) * 100
center = np.ones(25) * 50
flier_high = np.random.rand(10) * 100 + 100
flier_low = np.random.rand(10) * -100
data = np.concatenate((spread, center, flier_high, flier_low), 0)

# basic plot
plt.boxplot(data)

# notched plot
plt.figure()
plt.boxplot(data, 1)

# change outlier point symbols
plt.figure()
plt.boxplot(data, 0, 'gD')

# don't show outlier points
plt.figure()
plt.boxplot(data, 0, '')

# horizontal boxes
plt.figure()
plt.boxplot(data, 0, 'rs', 0)

# change whisker length
plt.figure()
plt.boxplot(data, 0, 'rs', 0, 0.75)

# fake up some more data
spread = np.random.rand(50) * 100
center = np.ones(25) * 40
flier_high = np.random.rand(10) * 100 + 100
flier_low = np.random.rand(10) * -100
d2 = np.concatenate((spread, center, flier_high, flier_low), 0)
data.shape = (-1, 1)
d2.shape = (-1, 1)
# data = concatenate( (data, d2), 1 )
# Making a 2-D array only works if all the columns are the
# same length.  If they are not, then use a list instead.
# This is actually more efficient because boxplot converts
# a 2-D array into a list of vectors internally anyway.
data = [data, d2, d2[::2, 0]]
# multiple box plots on one figure
plt.figure()
plt.boxplot(data)

plt.show()
